In [ ]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
import pyspark
from pyspark.sql import functions,SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
import gc
from pyspark.sql.types import BooleanType
# from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1681441215995_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# user pair
user_pair = spark.read.format('csv').options(header='False').load('s3://cs5344-twitter-project/input/user_pair').cache()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
user_pair_df = user_pair.select(col("_c1").alias("user_id_str"),
                 col("_c2").alias("cnt"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
user_pair_df.createOrReplaceTempView("user_pair_df")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt = spark.sql("""
    select user_id_str, sum(cnt) as re_cnt
    from user_pair_df
    group by user_id_str""")
retweets_cnt.createOrReplaceTempView("retweets_cnt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt_pokemon = spark.sql("""
select b.user_id_str, a.re_cnt
from retweets_cnt a
join pokemon b
on a.user_id_str = b.user_id_str
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:

output_bucket = 's3://cs5344-twitter-project/'
# pokemon_inf.toDF().write.mode('overwrite').csv(output_bucket+'pokemon_inf_24_aws') 
retweets_cnt_pokemon.write.format("csv").save(output_bucket+'user_ana/retweets_cnt_pokemon') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt_pokemon.show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------+
|        user_id_str| re_cnt|
+-------------------+-------+
| 987228577522376704| 1921.0|
|1441557548427927554| 1502.0|
|1441557548427927554| 1502.0|
| 909119962572242944| 2163.0|
|1235177017441529861|  896.0|
|         1149178590| 1302.0|
|1483999454244274178|  684.0|
|         2786677338| 5246.0|
|1470439914953011204| 1935.0|
|            5921162|  686.0|
|           19426551|  696.0|
|1034478294677250048|  855.0|
|            5162861| 2438.0|
|          130447415|13416.0|
|           34429906|  672.0|
|           96879107|  882.0|
|         2597010860| 1378.0|
|1240603506177748992|  869.0|
| 811765148595535872| 3501.0|
| 839280506931924992|  882.0|
|         4644145430|  667.0|
|         3095001124| 1839.0|
|          297169759| 1182.0|
|1302006102364745728| 1290.0|
|          425871040| 1072.0|
|             890891| 1212.0|
|1200355745600069635|  888.0|
|           19631395|  951.0|
|          307902310| 3293.0|
|          347831597| 1058.0|
|142804886

In [ ]:
input_bucket = 's3://cs5344-twitter-project'
test_pokemon_damping_sdf = spark.read.format('csv').options(header='False').load('s3://cs5344-twitter-project/test_pokemon_damping').cache()
pokemon = test_pokemon_damping_sdf.select(col("_c0").alias("user_id_str"))
pokemon.createOrReplaceTempView("pokemon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# retweets
input_bucket = 's3://cs5344-twitter-project'
conn_sdf = spark.read.format('json').options(header='True').load('s3://cs5344-twitter-project/input/retweeted_tweet_intermediate_full').cache()
conn_sdf.createOrReplaceTempView("conn_sdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt = spark.sql("""
    select user_id_str, count(1) as cnt
    from conn_sdf
    group by user_id_str""")
retweets_cnt.createOrReplaceTempView("retweets_cnt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt_pokemon = spark.sql("""select b.user_id_str, a.cnt from retweets_cnt a join pokemon b on a.user_id_str = b.user_id_str""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt_pokemon.show(50, truncate= False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---+
|user_id_str        |cnt|
+-------------------+---+
|987228577522376704 |5  |
|1441557548427927554|1  |
|1441557548427927554|1  |
|909119962572242944 |1  |
|2786677338         |6  |
|5921162            |9  |
|19426551           |7  |
|5162861            |13 |
|2597010860         |3  |
|96879107           |1  |
|839280506931924992 |2  |
|3095001124         |3  |
|1302006102364745728|13 |
|425871040          |1  |
|1200355745600069635|3  |
|890891             |1  |
|307902310          |3  |
|19631395           |3  |
|347831597          |1  |
|1428048861520547842|1  |
|1156269110545793024|1  |
|501558917          |1  |
|1365337673074933765|2  |
|1282594694095134720|3  |
|1328954190         |19 |
|1409917553514848257|4  |
|4920946291         |4  |
|26257166           |3  |
+-------------------+---+

In [ ]:
original_sdf = spark.read.format('json').options(header='True').load('s3://cs5344-twitter-project/input/original_tweet_intermediate_full').cache()
original_sdf.createOrReplaceTempView("original_sdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
original_sdf.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['connected_user_single', 'created_at', 'id_str', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'quoted_original_tweet_id_str', 'quoted_original_user_id_str', 'quoted_time', 'retweeted_original_tweet_id_str', 'retweeted_original_user_id_str', 'retweeted_time', 'text', 'text_hashtag', 'user_id_str']

In [ ]:
retweets_cnt_ori = spark.sql("""
    select user_id_str, count(1) as ori_cnt
    from original_sdf
    group by user_id_str""")
retweets_cnt_ori.createOrReplaceTempView("retweets_cnt_ori")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Column 'a.cnt' does not exist. Did you mean one of the following? [a.text, a.id_str, a.created_at, a.quoted_time, a.user_id_str, a.text_hashtag, b.user_id_str, a.retweeted_time, a.connected_user_single, a.in_reply_to_user_id_str, a.in_reply_to_status_id_str, a.quoted_original_user_id_str, a.quoted_original_tweet_id_str, a.retweeted_original_user_id_str, a.retweeted_original_tweet_id_str]; line 2 pos 22;
'Project [user_id_str#38, 'a.cnt]
+- Join Inner, (user_id_str#1359 = user_id_str#38)
   :- SubqueryAlias a
   :  +- SubqueryAlias conn_sdf
   :     +- View (`conn_sdf`, [connected_user_single#1346,created_at#1347,id_str#1348,in_reply_to_status_id_str#1349,in_reply_to_user_id_str#1350,quoted_original_tweet_id_str#1351,quoted_original_user_id_str#1352,quoted_time#1353,retweeted_original_tweet_id_str#1354,retweeted_original_user_id_str#1355,retweeted_time#1356,text#1357,text_hashtag#1358,user_id_str#1359])
   :        +- Relation [connected_user_single#1346,create

In [ ]:
retweets_cnt_pokemon.createOrReplaceTempView("retweets_cnt_pokemon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt_pokemon.show(50,  truncate= False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---+
|user_id_str        |cnt|
+-------------------+---+
|987228577522376704 |5  |
|1441557548427927554|1  |
|1441557548427927554|1  |
|909119962572242944 |1  |
|2786677338         |6  |
|5921162            |9  |
|19426551           |7  |
|5162861            |13 |
|2597010860         |3  |
|96879107           |1  |
|839280506931924992 |2  |
|3095001124         |3  |
|1302006102364745728|13 |
|425871040          |1  |
|1200355745600069635|3  |
|890891             |1  |
|19631395           |3  |
|307902310          |3  |
|347831597          |1  |
|1428048861520547842|1  |
|1156269110545793024|1  |
|501558917          |1  |
|1365337673074933765|2  |
|1282594694095134720|3  |
|1328954190         |19 |
|1409917553514848257|4  |
|4920946291         |4  |
|26257166           |3  |
+-------------------+---+

In [ ]:
# output_bucket = 's3://cs5344-twitter-project/'
# # pokemon_inf.toDF().write.mode('overwrite').csv(output_bucket+'pokemon_inf_24_aws') 
# pokemon_inf.write.format("csv").save(output_bucket+'pokemon_inf_24_aws') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt_pokemon_both = spark.sql("""
select b.user_id_str, a.cnt, b.ori_cnt
from retweets_cnt_pokemon a
join retweets_cnt_ori b
on a.user_id_str = b.user_id_str
""")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
retweets_cnt_pokemon_both.show(50,  truncate= False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---+-------+
|user_id_str        |cnt|ori_cnt|
+-------------------+---+-------+
|987228577522376704 |5  |87     |
|1441557548427927554|1  |1      |
|1441557548427927554|1  |1      |
|909119962572242944 |1  |263    |
|2786677338         |6  |170    |
|5921162            |9  |138    |
|19426551           |7  |165    |
|5162861            |13 |118    |
|96879107           |1  |79     |
|2597010860         |3  |17     |
|839280506931924992 |2  |3      |
|3095001124         |3  |37     |
|1302006102364745728|13 |3      |
|425871040          |1  |7      |
|1200355745600069635|3  |16     |
|890891             |1  |205    |
|19631395           |3  |100    |
|307902310          |3  |198    |
|347831597          |1  |16     |
|1428048861520547842|1  |61     |
|1156269110545793024|1  |27     |
|501558917          |1  |52     |
|1365337673074933765|2  |4      |
|1282594694095134720|3  |71     |
|1328954190         |19 |23     |
|1409917553514848257|4  |5      |
|4920946291   